<a href="https://colab.research.google.com/github/hamidriasat/Natural-Language-Processing/blob/master/Incident%20Severity%20Scoring/Incident_Project_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch-pretrained-bert


     |████████████████████████████████| 133kB 2.8MB/s 


In [ ]:
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-large-uncased')
model.eval()


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (

In [ ]:
!gdown --id 1H5YLUM-zsD7BbjMDO9iIJEjX5CQ-ols_

!unzip OverAllDataset


In [ ]:
import numpy as np
import torch


In [ ]:

def readLabels():
  full_labels= []
  with open( "OverAllDataset/Labels.txt" , encoding="cp1252") as content_file:
    for line in content_file:
      full_labels.append( int(line))
  return full_labels

def readData( ):
    full_date= []
    for i in range(1, 1147):
        with open( "OverAllDataset/"+ str(i) + ".txt" , encoding="cp1252") as content_file:
            fileData= content_file.read()
            templength= len(fileData.split() )
            if templength < 300:
                full_date.append(fileData )
            else:
                temp= ""
                convert= fileData.split()
                for j in range(0,300):
                    temp= temp+ " "+convert[j]
                full_date.append(temp )

    return full_date


data_X= readData()
data_Y= readLabels()

print(len(data_X))
print(len(data_Y))

1146
1146


In [ ]:
def extract_embed(data):
    embedings=np.zeros((len(data),1024))
    cnt=0
    for i in data:
        # print(len(i.split()))
        
        # Add the special tokens.
        marked_text = "[CLS] " +i+" [SEP]"

        # Split the sentence into tokens.
        tokenized_text = tokenizer.tokenize(marked_text)
        #print (len(tokenized_text)
        
        # Map the token strings to their vocabulary indeces.
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
        #print(indexed_tokens)
        # Mark each of the 22 tokens as belonging to sentence "1".
        segments_ids = [1] * len(tokenized_text)
        # Convert inputs to PyTorch tensors
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])
        
        # Predict hidden states features for each layer
        with torch.no_grad():
            encoded_layers, _ = model(tokens_tensor, segments_tensors)
        # Concatenate the tensors for all layers. We use `stack` here to
        # create a new dimension in the tensor.
        token_embeddings = torch.stack(encoded_layers, dim=0)
        # Remove dimension 1, the "batches".
        token_embeddings = torch.squeeze(token_embeddings, dim=1)

        # Swap dimensions 0 and 1.
        token_embeddings = token_embeddings.permute(1,0,2)

        # `token_vecs` is a tensor with shape [22 x 768]
        token_vecs = encoded_layers[11][0]

        # Calculate the average of all 22 token vectors.
        sentence_embedding = torch.mean(token_vecs, dim=0)
        
        embedings[cnt]=sentence_embedding
        cnt+=1
        
    return embedings


In [ ]:
X= extract_embed( data_X )

In [ ]:
print( X.shape)
print( type(X))
Y= np.array(data_Y)
print( Y.shape)
print( type(Y))

(1146, 1024)
<class 'numpy.ndarray'>
(1146,)
<class 'numpy.ndarray'>


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20)



In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

classifier = SVC(kernel='linear')
# classifier = LogisticRegression(max_iter= 1000)
classifier.fit(X_train, y_train)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
y_pred = classifier.predict(X_test)


In [ ]:
# sing_pred=svclassifier.predict(X_test[0].reshape(1, -1))


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[28 17  2  1  0  2]
 [13 36  7  4  3  1]
 [ 6  8  5  0  2  0]
 [ 5  6  3 33  2  0]
 [ 2  3  2  9 13  4]
 [ 0  0  0  2  8  3]]
              precision    recall  f1-score   support

           0       0.52      0.56      0.54        50
           1       0.51      0.56      0.54        64
           2       0.26      0.24      0.25        21
           3       0.67      0.67      0.67        49
           4       0.46      0.39      0.43        33
           5       0.30      0.23      0.26        13

    accuracy                           0.51       230
   macro avg       0.46      0.44      0.45       230
weighted avg       0.51      0.51      0.51       230



In [ ]:
print(accuracy_score(y_test, y_pred))